# Prezentacja działania systemu automatyzacji (system warunkowy)

W bazie danych została zapisana automatyzacja oczekująca na temperaturę `5.5` z urządzenia o adresie MAC `00:00:00:00:00:01`. System, po otrzymaniu takiej wartości, powinien wysłać do urządzenia o adresie MAC `00:00:00:00:00:02` wiadomość MQTT o wartości `1`. Symulowany jest tu moduł `mqtt-listener`, który po odebraniu takiej wartości wysyła zapytanie HTTP do API. Jednocześnie, zasubskrybowany został temat MQTT odpowiedni dla urządzenia sterowanego przez automatyzację.

Pseudowarunek *IF this THEN that*:

```
IF
    Temperature@00:00:00:00:00:01 EQ 5.5
THEN
    Switch@00:00:00:00:00:02 SET TO 1
```

In [1]:
const mqtt = require("mqtt");
const axios = require("axios");

In [2]:
var client = mqtt.connect("mqtt://vps.sokoloowski.pl:1883");

client.on("connect", function () {
    client.subscribe("/studio/00:00:00:00:00:02/Switch/set");
    console.log("Client has subscribed successfully");
});

client.on("message", function (topic, payload, packet) {
    console.log(`Topic: ${topic}, Message: ${payload.toString()}`);
});

console.log("Automation example")

Automation example
Client has subscribed successfully


Po wysłaniu zapytania HTTP, powinniśmy dostać wiadomość MQTT na zasubskrybowanym temacie oraz odpowiedź z API.

In [3]:
console.log("Sending request...");
axios
    .post("http://127.0.0.1:8000/api/sensor/mqtt", {
        username: "jkowalski@example.com",
        device: "00:00:00:00:00:01",
        value: 5.5,
        type: "Temperature",
        received: new Date().getTime(),
    })
    .then(function (response) {
        console.log(`HTTP response: ${response.data.message}`);
    })
    .catch(function (error) {
        console.log(`Error: ${error}`);
    });

Sending request...


Promise { <pending> }

Topic: /studio/00:00:00:00:00:02/Switch/set, Message: 1
HTTP response: OK
